In [1]:
import re
import json
import pandas as pd
from collections import defaultdict

import json
from json import dumps

## merge data sources

In [2]:
with open('dienstleistungen.json') as data_file:    
    data = json.load(data_file)

In [3]:
dienstleistungen = pd.DataFrame(data['data'])

In [4]:
# extracted Leika Gruppe, Leika Kennung, Leika Verrichtung & Leika Synonyme from d115_to_leika.csv
leika = pd.DataFrame.from_csv('data.tsv', sep='\t').reset_index()

In [5]:
# parse ID as string
leika[['D115 ID']] = leika[['D115 ID']].astype(str)
dienstleistungen[['id']] = dienstleistungen[['id']].astype(str)

In [6]:
result = pd.merge(dienstleistungen, leika, left_on='id', right_on='D115 ID')

In [7]:
result.drop('D115 ID', axis=1, inplace=True)
result.drop('Leika Nummer', axis=1, inplace=True)

In [8]:
result.head()

,appointment,authorities,description,fees,forms,id,legal,leika,links,locations,...,relation,representation,requirements,residence,responsibility,responsibility_all,Leika Gruppe,Leika Kennung,Leika Verrichtung,Leika Synonyme
0,{'link': 'https://service.berlin.de/terminvere...,[{'name': 'Bezirksamt Charlottenburg - Wilmers...,"<ul class=""list""><li>Sie geben eine Berliner W...",gebührenfrei; dies gilt auch für die Abmeldebe...,"[{'name': 'Abmeldung bei der Meldebehörde', 'l...",120335,"[{'name': 'Bundesmeldegesetz', 'link': 'http:/...",99115005070000,[{'name': 'Informationen zum Bundesmeldegesetz...,"[{'location': '122231', 'appointment': {'link'...",...,{'root_topic': '324835'},0,"[{'name': 'Abmeldung bei der Meldebehörde', 'l...",0,Diese Dienstleistung erhalten Sie in allen Ber...,True,Wohnsitz,Wohnsitz,Abmeldung,Abmeldung | Wohnungsabmeldung | Wegzug | abmel...
1,{'link': ''},[{'name': 'Bezirksamt Charlottenburg - Wilmers...,Wohngeld kann Ihnen angemessenes und familien-...,keine,"[{'name': 'Antrag auf Mietzuschuss', 'link': '...",120656,"[{'name': 'Wohngeldgesetz', 'link': 'http://ww...",11107023017900,"[{'name': 'Wohngeldreform zum 1. Januar 2016',...","[{'location': '122231', 'appointment': {'link'...",...,{'root_topic': '324831'},0,"[{'name': 'Antragsformular', 'link': 'https://...",0,Den ausgefüllten Antrag auf Bewilligung von Wo...,false,Sozialleistungen,Wohngeld,Bewilligung,Förderung | Wohnhilfe | Wohnen | Wohngeld | Wo...
2,{'link': 'https://service.berlin.de/terminvere...,[{'name': 'Bezirksamt Charlottenburg - Wilmers...,Befindet sich die Hauptwohnung in Berlin und d...,"bis 11,00 Euro",[],120658,[{'name': 'Straßenverkehrszulassungsordnung - ...,11036008014900,[],"[{'location': '122231', 'appointment': {'link'...",...,{'root_topic': '324803'},0,[{'name': 'Fahrzeugschein / Zulassungsbeschein...,0,Die Dienstleistung kann bei der Zulassungsbehö...,True,Fahrzeugzulassung,Kraftfahrzeug,Meldung,Fahrzeugscheinänderungen | Zulassungsbescheini...
3,{'link': ''},[{'name': 'Bezirksamt Charlottenburg - Wilmers...,Wohngeld kann Ihnen angemessenes und familien-...,keine,"[{'name': 'Antrag auf Lastenzuschuss', 'link':...",120665,"[{'name': 'Wohngeldgesetz', 'link': 'http://ww...",11107023017901,"[{'name': 'Wohngeldreform zum 1. Januar 2016',...","[{'location': '122231', 'appointment': {'link'...",...,{'root_topic': '324831'},0,"[{'name': 'Antragsformular ', 'link': 'https:/...",0,Den ausgefüllten Antrag auf Lastenzuschuss und...,false,Sozialleistungen,Wohngeld,Bewilligung,Wohnung | Förderung | Wohngeld | Wohnhilfe | E...
4,{'link': ''},[{'name': 'Bezirksamt Charlottenburg - Wilmers...,Mit einem Wohnberechtigungsschein (WBS) können...,Gebührenfrei,[{'name': 'Antrag auf einen Wohnberechtigungss...,120671,[{'name': 'Gesetz über die soziale Wohnraumför...,11107022900900,[],"[{'location': '122231', 'appointment': {'link'...",...,{'root_topic': '324831'},0,[{'name': 'Antrag auf einen Wohnberechtigungss...,0,"Wohnungsamt des Bezirks, in dem Sie wohnen. Wo...",false,Sozialleistungen,Wohnberechtigungsschein,WBS - Antragsannahme,Sozialwohnung | Zinsbescheinigung | Wohnraum |...


## create nested JSON from grouped dataframe

In [10]:
json_data = defaultdict(lambda: defaultdict(list))
result['Leika Verrichtung'] = result['Leika Verrichtung'].astype(str)

for index, row in result.iterrows():
    # firebase parsing
    kennung = re.sub('[\$#\[\]\/\.]', ' ', row['Leika Kennung'])
    verrichtung = re.sub('[\$#\[\]\/\.]', ' ', row['Leika Verrichtung'])
    
    service = row.to_dict()
    
    json_data[kennung][verrichtung].append(service)
    
    
output = {}
output['tree'] = json_data


### add standorte.json to data structure

In [15]:
with open('standorte.json') as data_file:    
    data = json.load(data_file)

In [16]:
output['standorte'] = data['data']

In [17]:
json_out = dumps(output)

f = open('base.json','w')
f.write(json_out)
f.close()